# Data fold

In [ ]:
index = 3

# Code & Data

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
!rm -rf sample_data
!unzip -q drive/MyDrive/GWC21/train.zip -d ./
!unzip -q drive/MyDrive/GWC21/test.zip -d ./
!cp drive/MyDrive/GWC_data/submission.csv ./
!cp drive/MyDrive/GWC_data/{index}.zip ./

In [ ]:
!unzip -q {index}.zip -d ./

In [ ]:
import shutil

with open(f'{index}/val.txt') as f:
  for line in f:
    file = 'train/' + line.strip()
    shutil.copyfile(file, f'{index}/images/val/'+line.strip())
with open(f'{index}/train.txt') as f:
  for line in f:
    file = 'train/' + line.strip()
    shutil.copyfile(file, f'{index}/images/train/'+line.strip())

In [ ]:
!pip uninstall -y PyYAML
!pip install PyYAML==5.3.1
!pip install ensemble_boxes

In [ ]:
import yaml

result = {'train': f'../{str(index)}/images/train', 'val': f'../{str(index)}/images/val', 'nc': 1, 'names': ['wheat']}
with open('GWC_YOLO/data/custom.yaml', 'w') as f:
  dump = yaml.dump(result, default_flow_style=False)
  f.write(dump)

In [ ]:
!cd GWC_YOLO && python train.py --name 4fold{index}_800_45 --img 800 --batch 8 --epochs 45 --data custom.yaml --weights yolov5x.pt --cache-images

In [ ]:
!cp -r GWC_YOLO/runs/train/4fold3_800_45 drive/MyDrive/

# Pseudo labeling

In [ ]:
import os, shutil

def run_pseudo_labeling(weights, base_dir, generate=True):
  if generate:
    if os.path.exists('pseudo'):
      shutil.rmtree('pseudo')
    os.mkdir('pseudo')
    os.mkdir('pseudo/images')
    os.mkdir('pseudo/labels')
    os.mkdir('pseudo/images/train')
    os.mkdir('pseudo/labels/train')
    !cd GWC_YOLO && python detect.py --img-size 800 --weights {weights} --source ../test --augment --nosave --save-txt --conf-thres 0.5

    files = os.listdir(base_dir)
    for file in files:
      shutil.copyfile(os.path.join(base_dir, file), 'pseudo/labels/train/'+file)
    images = os.listdir('test')
    for image in images:
      shutil.copyfile(os.path.join('test', image), 'pseudo/images/train/'+image)

  result = {'train': [f'../{str(index)}/images/train', '../pseudo/images/train'], 'val': f'../{str(index)}/images/val', 'nc': 1, 'names': ['wheat']}
  with open('GWC_YOLO/data/custom_pseudo.yaml', 'w') as f:
    dump = yaml.dump(result, default_flow_style=False)
    f.write(dump)
  !cd GWC_YOLO && python train.py --name 4fold{index}_800_new-wbf-pseudo-final_15_lr_0100 --img 800 --batch 8 --epochs 15 --data custom_pseudo.yaml --weights {weights}  --hyp data/hyp.final.yaml --cache

In [ ]:
run_pseudo_labeling('../drive/MyDrive/runs/train/round2/4fold3_800_45_0.773/weights/best.pt', 'GWC_YOLO/runs/detect/4fold3_800_45_0.773-best_conf-0.5_aug-True/labels')
# run_pseudo_labeling('../drive/MyDrive/4fold0_35_0.778/weights/best.pt', 'GWC_YOLO/runs/detect/4fold0_35_0.778-best_conf-0.5_aug-True/labels')

In [ ]:
!cd GWC_YOLO && python train.py --name 4fold{index}_800_new-wbf-pseudo-final_15_lr_0100 --img 800 --batch 8 --epochs 15 --data custom_pseudo.yaml --weights ../drive/MyDrive/runs/train/round2/4fold3_800_45_0.773/weights/best.pt  --hyp data/hyp.final.yaml --cache

In [ ]:
!cp -r GWC_YOLO/runs/train/4fold3_800_new-wbf-pseudo-final_15_lr_0100 drive/MyDrive/runs/train/round2

# Save to submission

In [ ]:
# !pip install ensemble_boxes
!cd GWC_YOLO && python detect.py --img-size 800 --weights ../drive/MyDrive/runs/train/round2/4fold3_800_new-wbf-pseudo-final_15_lr_0100/weights/12.pt --name 12 --source ../test --augment --nosave --save-csv --conf-thres 0.5

In [ ]:
!pip install aicrowd-cli
API_KEY = "" 
!aicrowd login --api-key $API_KEY

In [ ]:
!aicrowd submission create -c global-wheat-challenge-2021 -f GWC_YOLO/runs/detect/12/submission.csv